In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Importing necessary libraries


In [ ]:
import numpy as np
import pandas as pd
import tracemalloc
import time
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_recall_fscore_support

## Algorithm Directory path

In [ ]:
# All the datasets are loaded from this directory and
# All the results are stored in this directory
directoryPath = '/content/drive/MyDrive/Colab Notebooks/Data Mining Assignments/Classification/'

## Node Class

In [ ]:
class Node:
  def __init__(self):
    self.parent = None
    self.children = dict()
    self.classLabel = None
    self.attributeIdx = None
    self.attributeType = None
    self.splitPoint = None
    self.majorityClass = None

  def get_majorityClass(self):
    return self.majorityClass
  
  def set_majorityClass(self, majorityClass):
    self.majorityClass = majorityClass

  def get_parent(self):
    return self.parent
  
  def set_parent(self, parent):
    self.parent = parent

  def get_classLabel(self):
    return self.classLabel

  def set_classLabel(self, label):
    self.classLabel = label
  
  def get_children(self):
    return self.children

  def get_single_children(self, key):
    return self.children[key]

  def get_attributeIdx(self):
    return self.attributeIdx

  def set_attributeIdx(self, attributeIdx):
    self.attributeIdx = attributeIdx

  def get_attributeType(self):
    return self.attributeType

  def set_attributeType(self, attributeType):
    self.attributeType = attributeType

  def get_splitPoint(self):
    return self.splitPoint

  def set_splitPoint(self, splitPoint):
    self.splitPoint = splitPoint

  def print_node(self):
    print(self)
    print("majorityClass = {}".format(self.majorityClass))
    print("parent = {}".format(self.parent))
    print("children = {}".format(self.children))
    print("attributeIdx = {}".format(self.attributeIdx))
    print("attributeType = {}".format(self.attributeType))
    print("classLabel = {}".format(self.classLabel))
    print("splitPoint = {}".format(self.splitPoint))

  def insert_child(self, childNode, edge):
    self.children[edge] = childNode

  def delete_child(self, edge):
    del self.children[edge]

## Decision Tree Classifier


In [ ]:
class DecisionTree:

  def __init__(self, X, Y, selectionMeasure = 'gain', discreteThreshold = 15):
    self.X = X
    self.Y = Y
    self.discreteThreshold = discreteThreshold
    self.selectionMeasure = selectionMeasure
    self.attributeInfo = self.get_attribute_info()
    self.root = self.build_decision_tree(self.X, self.Y, self.attributeInfo)
  
  def get_attribute_info(self):
    attributeInfo = []
    for i in range(self.X.shape[1]):
      column = self.X[:, i]
      distinctValues = np.unique(column)
      attributeType = 'discrete' if len(distinctValues) <= self.discreteThreshold else 'continuous'
      attributeInfo.append({
          'idx' : i,
          'type' : attributeType,
          'distinctValues' : distinctValues if attributeType == 'discrete' else None
      })
    
    return np.array(attributeInfo)

  def calculate_entropy(self,Y):
    if not len(Y):
      return 0.0
    _, classCounts = np.unique(Y, return_counts=True)
    classProbs = classCounts/np.sum(classCounts)
    entropy = (-1) * np.sum(classProbs * np.log2(classProbs))
    return entropy

  def calculate_entropy_for_particular_attrbute(self, X, Y, value):
    return None

  def calculate_gain(self):
    return None

  def calculate_split_information(self):
    return None

  def calculate_gain_ratio(self):
    return None

  def calculate_gini_index(self):
    return None

  def find_best_splitting_criterion_using_gain(self, X, Y, attributeInfo):
    maxGain = -np.inf
    splittingAttributeIdx = None
    splitPoint = None

    for i in range(len(attributeInfo)):
      gain = None
      bestSplitPoint = None
      
      if attributeInfo[i]['type'] == 'discrete':
        totalEntropy = 0
        for value in attributeInfo[i]['distinctValues']:
          entropy = ((X[:, i] == value).sum()/len(X[:, i])) * self.calculate_entropy(Y[X[:, i] == value])
          totalEntropy += entropy
        gain = self.calculate_entropy(Y) - totalEntropy
        # print("Gain for {}th attribute = {}\n".format(attributeInfo[i]['idx'], gain))

      else: 
        sortedColumn = np.sort(X[:, i])
        minEntropy = np.inf
        for j in range(1, len(sortedColumn)):
          midpoint = (sortedColumn[j-1] + sortedColumn[j])/2.0
          lessEqualEntropy = ((X[:, i] <= midpoint).sum()/len(X[:, i])) * self.calculate_entropy(Y[X[:, i] <= midpoint])
          greaterEntropy = ((X[:, i] > midpoint).sum()/len(X[:, i])) * self.calculate_entropy(Y[X[:, i] > midpoint])
          totalEntropy = lessEqualEntropy + greaterEntropy
          if totalEntropy < minEntropy:
            minEntropy = totalEntropy
            bestSplitPoint = midpoint

        gain = self.calculate_entropy(Y) - minEntropy

      if gain > maxGain:
        maxGain = gain
        splitPoint = bestSplitPoint
        splittingAttributeIdx = attributeInfo[i]['idx']

    return splittingAttributeIdx, splitPoint

  def find_best_splitting_criterion_using_gini_index(self, X, Y, attributeInfo):
    return None, None

  def find_best_splitting_criterion_using_gain_ratio(self, X, Y, attributeInfo):
    maxGainRatio = -np.inf
    splittingAttributeIdx = None
    splitPoint = None

    for i in range(len(attributeInfo)):
      gain = None
      bestSplitPoint = None
      
      if attributeInfo[i]['type'] == 'discrete':
        totalEntropy = 0
        totalSplitInfo = 0
        for value in attributeInfo[i]['distinctValues']:
          probability = ((X[:, i] == value).sum()/len(X[:, i]))
          entropy =  probability * self.calculate_entropy(Y[X[:, i] == value])
          splitInfo = probability * np.log2(probability) if probability != 0 else 0 
          totalSplitInfo += splitInfo 
          totalEntropy += entropy
        gain = self.calculate_entropy(Y) - totalEntropy
        totalSplitInfo = (-1) * totalSplitInfo
        gainRatio = gain/totalSplitInfo
        # print("Gain for {}th attribute = {}\n".format(attributeInfo[i]['idx'], gain))

      else: 
        sortedColumn = np.sort(X[:, i])
        minEntropy = np.inf
        effectiveSplitInfo = None
        for j in range(1, len(sortedColumn)):
          midpoint = (sortedColumn[j-1] + sortedColumn[j])/2.0
          lessEqualProbability = ((X[:, i] <= midpoint).sum()/len(X[:, i]))
          lessEqualEntropy = lessEqualProbability * self.calculate_entropy(Y[X[:, i] <= midpoint])
          lessEqualSplitInfo = lessEqualProbability * np.log2(lessEqualProbability) if lessEqualProbability != 0 else 0

          greaterProbability = ((X[:, i] > midpoint).sum()/len(X[:, i]))
          greaterEntropy = greaterProbability * self.calculate_entropy(Y[X[:, i] > midpoint])
          greaterSplitInfo = greaterProbability * np.log2(greaterProbability) if greaterProbability != 0 else 0

          totalEntropy = lessEqualEntropy + greaterEntropy
          totalSplitInfo = lessEqualSplitInfo + greaterSplitInfo

          # if lessEqualSplitInfo == 0 or greaterSplitInfo == 0:
          #   print("sortedColumn[j-1] = {} + sortedColumn[j] = {}".format(sortedColumn[j-1] , sortedColumn[j]))
          #   print("midpoint = {}".format(midpoint))
          #   print("colum = {}".format(X[:, i]))
          #   print("lessEqualProbability = {} + greaterProbability = {}".format( lessEqualProbability , greaterProbability))
          #   print("lessEqualSplitInfo = {} + greaterSplitInfo = {}".format( lessEqualSplitInfo , greaterSplitInfo))

          if totalEntropy < minEntropy:
            minEntropy = totalEntropy
            bestSplitPoint = midpoint
            effectiveSplitInfo = totalSplitInfo

        gain = self.calculate_entropy(Y) - minEntropy
        effectiveSplitInfo = (-1) * effectiveSplitInfo
        gainRatio = gain/effectiveSplitInfo

      if gainRatio > maxGainRatio:
        maxGainRatio = gainRatio
        splitPoint = bestSplitPoint
        splittingAttributeIdx = attributeInfo[i]['idx']

    return splittingAttributeIdx, splitPoint

  def generate_edge_from_partition(self, node, majorityClass, edgeValue, partitionedX, partitionedY, attributeInfo):
    if not len(partitionedY):
      childNode = Node()
      childNode.set_parent(node)
      childNode.set_classLabel(majorityClass)
      node.insert_child(childNode, edgeValue)
    
    else:
      childNode = self.build_decision_tree(partitionedX, partitionedY, attributeInfo)
      childNode.set_parent(node)
      node.insert_child(childNode, edgeValue)

  def build_decision_tree(self, X, Y, attributeInfo):
    node = Node()
    distinctClasses, classCounts = np.unique(Y, return_counts=True)
    majorityClass = distinctClasses[np.argmax(classCounts)]
    node.set_majorityClass(majorityClass)

    if len(distinctClasses) == 1:
      node.set_classLabel(distinctClasses[0])
      return node
    
    if len(attributeInfo) == 0: 
      node.set_classLabel(majorityClass)
      return node
    
    if self.selectionMeasure == 'gain':
      splittingAttributeIdx, splitPoint = self.find_best_splitting_criterion_using_gain(X, Y, attributeInfo)
    elif self.selectionMeasure == 'gain_ratio':
      splittingAttributeIdx, splitPoint = self.find_best_splitting_criterion_using_gain_ratio(X, Y, attributeInfo)
    else:
      splittingAttributeIdx, splitPoint = self.find_best_splitting_criterion_using_gini_index(X, Y, attributeInfo)

    # print("splittingAttributeIdx = ", splittingAttributeIdx)
    idx = np.array([k for k in range(len(attributeInfo)) if attributeInfo[k]['idx'] == splittingAttributeIdx]).item()
    node.set_attributeType(attributeInfo[idx]['type'])
    node.set_attributeIdx(splittingAttributeIdx)
    node.set_splitPoint(splitPoint)

    if attributeInfo[idx]['type'] == 'discrete':
      if self.selectionMeasure != 'gini_index':
        for value in attributeInfo[idx]['distinctValues']:
          partitionedX = X[X[:, idx] == value]
          partitionedX = np.delete(partitionedX, idx, 1)
          partitionedAttributeInfo = np.delete(attributeInfo, idx)
          partitionedY = Y[X[:, idx] == value]

          self.generate_edge_from_partition(node, majorityClass, value, partitionedX, partitionedY, partitionedAttributeInfo)
          
    else:
      leftPartitionedX = X[X[:, idx] <= splitPoint]
      leftPartitionedY = Y[X[:, idx] <= splitPoint]
      rightPartitionedX = X[X[:, idx] > splitPoint]
      rightPartitionedY = Y[X[:, idx] > splitPoint]

      self.generate_edge_from_partition(node, majorityClass, 'left', leftPartitionedX, leftPartitionedY, attributeInfo)
      self.generate_edge_from_partition(node, majorityClass, 'right', rightPartitionedX, rightPartitionedY, attributeInfo)

    return node

  def recursive_print_decision_tree(self, node):
    if not len(node.children):
      print("Class label : {}".format(node.get_classLabel()))
      print()
      return
    
    print("attributeIdx = {}".format(node.attributeIdx))
    print("attributeType = {}".format(node.attributeType))
    print("splitPoint = {}".format(node.splitPoint))
    print()

    for key, child in node.children.items():
      print("-------> edge = {}".format(key))
      print()
      self.recursive_print_decision_tree(child)

  def print_decision_tree(self):
    self.recursive_print_decision_tree(self.root)

  def recursive_predict_one(self, x, node):
    if not len(node.children):
      return node.get_classLabel()
    
    attributeIdx = node.get_attributeIdx()
    if node.get_attributeType() == 'discrete':
      if x[attributeIdx] not in node.get_children():
        return node.get_majorityClass()
      return self.recursive_predict_one(x, node.get_single_children(x[attributeIdx]))
    else:
      if x[attributeIdx] <= node.get_splitPoint():
        return self.recursive_predict_one(x, node.get_single_children('left'))
      else:
        return self.recursive_predict_one(x, node.get_single_children('right'))

  def predict_one(self, x):
    return self.recursive_predict_one(x, self.root)

  def predict(self, XTest):
    YPred = []
    for x in XTest:
      YPred.append(self.predict_one(x))
    return np.array(YPred)

##Running on Sample datasets


In [ ]:
# filePath = directoryPath + 'sampleDataset-1.csv'
# df = pd.read_csv(filePath, sep=",", header=None)
# dfX = df.iloc[:,:-1]
# dfY = df.iloc[:,-1]
# # print(dfX.head())
# # print(dfY.head())

# X = dfX.to_numpy()
# Y = np.squeeze(dfY.to_numpy())
# # print('instances = {}, features= {} '.format(X.shape[0], X.shape[1]))

# decisionTree = DecisionTree(X, Y,selectionMeasure = 'gain_ratio',  discreteThreshold = 15)
# # decisionTree.print_decision_tree()
# YPred = decisionTree.predict(X)

# # print(YPred)
# print((Y == YPred).sum())

In [ ]:
# filePath = directoryPath + 'sampleDataset-2.csv'
# df = pd.read_csv(filePath, sep=",", header=None)
# dfX = df.iloc[:,:-1]
# dfY = df.iloc[:,-1]
# # print(dfX.head())
# # print(dfY.head())

# X = dfX.to_numpy()
# Y = np.squeeze(dfY.to_numpy())
# # print('instances = {}, features= {} '.format(X.shape[0], X.shape[1]))

# decisionTree = DecisionTree(X, Y,selectionMeasure = 'gain_ratio',  discreteThreshold = 15)
# # decisionTree.print_decision_tree()
# YPred = decisionTree.predict(X)
# print((Y == YPred).sum())

# XTest = np.array([['senior', 'high', 'no', 'eee'], ['youth', 'high', 'eee', 'fair'], ['Buira_orko', 'high', 'no', 'fair']])
# YPred = decisionTree.predict(XTest)
# print(YPred)

In [ ]:
# from sklearn.datasets import load_iris
# X, Y = load_iris(return_X_y= True)
# Y = np.squeeze(Y)
# # print('instances = {}, features= {} '.format(X.shape[0], X.shape[1]))

# decisionTree = DecisionTree(X, Y,selectionMeasure = 'gain',  discreteThreshold = 15)
# # decisionTree.print_decision_tree()
# YPred = decisionTree.predict(X)
# print(len(Y))
# print((Y == YPred).sum())

In [ ]:
# from sklearn.datasets import load_wine
# X, Y = load_wine(return_X_y= True)
# Y = np.squeeze(Y)
# # print('instances = {}, features= {} '.format(X.shape[0], X.shape[1]))

# decisionTree = DecisionTree(X, Y,selectionMeasure = 'gain',  discreteThreshold = 15)
# # decisionTree.print_decision_tree()
# YPred = decisionTree.predict(X)
# print(len(Y))
# print((Y == YPred).sum())

In [ ]:
# from sklearn.datasets import load_digits
# X, Y = load_digits(return_X_y= True)
# Y = np.squeeze(Y)
# # print('instances = {}, features= {} '.format(X.shape[0], X.shape[1]))

# decisionTree = DecisionTree(X, Y,selectionMeasure = 'gain',  discreteThreshold = 15)
# # decisionTree.print_decision_tree()
# YPred = decisionTree.predict(X)
# print(len(Y))
# print((Y == YPred).sum())

In [ ]:
# filePath = 'breast-cancer.csv'
# df = pd.read_csv(filePath, sep=",", header=None)
# dfX = df.iloc[:,1:]
# dfY = df.iloc[:,0]
# # print(dfX.head())
# # print(dfY.head())

# X = dfX.to_numpy()
# Y = np.squeeze(dfY.to_numpy())
# # print('instances = {}, features= {} '.format(X.shape[0], X.shape[1]))

# decisionTree = DecisionTree(X, Y,selectionMeasure = 'gain',  discreteThreshold = 15)
# decisionTree.print_decision_tree()
# YPred = decisionTree.predict(X)
# print(len(Y))
# print((Y == YPred).sum())

# # XTest = np.array([['senior', 'high', 'no', 'eee'], ['youth', 'high', 'eee', 'fair'], ['Buira_orko', 'high', 'no', 'fair']])
# # YPred = decisionTree.predict(XTest)
# # print(YPred)

attributeIdx = 5
attributeType = discrete
splitPoint = None

-------> edge = 1

attributeIdx = 2
attributeType = discrete
splitPoint = None

-------> edge = 0-4

Class label : no-recurrence-events

-------> edge = 10-14

Class label : no-recurrence-events

-------> edge = 15-19

attributeIdx = 0
attributeType = discrete
splitPoint = None

-------> edge = 20-29

Class label : no-recurrence-events

-------> edge = 30-39

attributeIdx = 6
attributeType = discrete
splitPoint = None

-------> edge = left

Class label : no-recurrence-events

-------> edge = right

Class label : recurrence-events

-------> edge = 40-49

Class label : no-recurrence-events

-------> edge = 50-59

Class label : no-recurrence-events

-------> edge = 60-69

Class label : no-recurrence-events

-------> edge = 70-79

Class label : recurrence-events

-------> edge = 20-24

attributeIdx = 1
attributeType = discrete
splitPoint = None

-------> edge = ge40

attributeIdx = 7
attributeType = discrete
splitPoint = None

--